# Data Preprocessing and Exploration

Given the intial dataset provided by kaggle we will adjust the data to best fit the algorithms we will try later:

1. LDA: The data is provided with an entire image and bounding boxes, we will extract sections of pneumonia from the data to extract features about the pneumonia rich regions. 
2. RCNN: We will analyze the mean and variance of our images to inform the generation of alternative images generated by transforming the image (scale,rotate, etc..) 



## Import Data

In [4]:
import os
import sys

import random
import math
import numpy as np
import matplotlib.pyplot as plt
import json
import pydicom
import pandas as pd 
import glob
import cv2


# Root directory of the project
ROOT_DIR = os.path.abspath('./data')

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')


train_dicom_dir = os.path.join(ROOT_DIR, 'stage_2_train_images')
test_dicom_dir = os.path.join(ROOT_DIR, 'stage_2_test_images')

ROOT_DIR

'/Users/wilsontang/Desktop/ML_PneumDetect_2018/data'

## Helper Functions

In [2]:
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations

In [3]:
class DetectorDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # Add classes
        self.add_class('pneumonia', 1, 'Lung Opacity')
   
        # add images 
        for i, fp in enumerate(image_fps):
            annotations = image_annotations[fp]
            self.add_image('pneumonia', image_id=i, path=fp, 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        ds = pydicom.read_file(fp)
        image = ds.pixel_array
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                if a['Target'] == 1:
                    x = int(a['x'])
                    y = int(a['y'])
                    w = int(a['width'])
                    h = int(a['height'])
                    mask_instance = mask[:, :, i].copy()
                    cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
                    mask[:, :, i] = mask_instance
                    class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

NameError: name 'utils' is not defined

In [ ]:
anns = pd.read_csv(os.path.join(ROOT_DIR, 'stage_2_train_labels.csv'))
anns.head()

In [ ]:
image_fps, image_annotations = parse_dataset(train_dicom_dir, anns=anns)

In [14]:
# helper function to calculate IoU
def iou(box1, box2):
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    if len(boxes_true) == 0 and len(boxes_pred) == 0:
        return None
    
    assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
    if len(boxes_pred):
        assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
        # sort boxes_pred by scores in decreasing order
        boxes_pred = boxes_pred[np.argsort(scores)[::-1], :]
    
    map_total = 0
    
    # loop over thresholds
    for t in thresholds:
        matched_bt = set()
        tp, fn = 0, 0
        for i, bt in enumerate(boxes_true):
            matched = False
            for j, bp in enumerate(boxes_pred):
                miou = iou(bt, bp)
                if miou >= t and not matched and j not in matched_bt:
                    matched = True
                    tp += 1 # bt is matched for the first time, count as TP
                    matched_bt.add(j)
            if not matched:
                fn += 1 # bt has no match, count as FN
                
        fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
        m = tp / (tp + fn + fp)
        map_total += m
    
    return map_total / len(thresholds)

In [19]:
import csv
# directory
label_file_path = os.path.abspath('./data/stage_2_train_labels.csv')
predictions_file_path = os.path.abspath('./data/stage_2_train_predictions.csv')

# load data 
#with open(label_file_path, newline='') as csvfile:
#     data_labels = list(csv.reader(csvfile))
# with open(predictions_file_path, newline='') as csvfile:
#     data_predictions = list(csv.reader(csvfile))

data_labels = pd.read_csv(os.path.join(ROOT_DIR, 'stage_2_train_labels.csv'))
data_predictions = pd.read_csv(os.path.join(ROOT_DIR, 'stage_2_train_labels.csv'))
#data_labels.fillna(0);
#data_predictions.fillna(0);

In [20]:
data_labels


,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1


In [47]:
data_predictions.keys()
data_predictions.iloc[1]['patientId']
# for index,row in data_predictions.iterrows():
#     print(index)
#     print(row)

'00313ee0-9eaa-42f4-b0ab-c148ed3241cd'

In [56]:
#Function to restore by patient in dictionary
def list_by_patient(data_set):
    data = dict()
    for index,row in data_predictions.iterrows():
        patient = data_predictions.iloc[index]['patientId']
#         if patient in data:
#             data.update(patient = np.concatenate(data[patient],(np.array(data_predictions.iloc[index][1:5]))))
#         else:
        data[patient] = np.array(data_predictions.iloc[index][1:5])
    return data

labels_list = list_by_patient(data_labels)
predictions_list = list_by_patient(data_predictions)

In [ ]:
# calculate accuracy 
count = 0
sum = 0.0
for image in 

for k in data_predictions.keys():
    print(data_labels[k], data_predictions[k]['bbox'], data_predictions[k]['score'])
    if data_labels[k] == []:
        if pd[k]['bbox'] == []:
            continue
        else:
            count += 1
    else:
        sum += map_iou(np.array(data_labels[k]), np.array(data_predictions[k]['bbox']), np.array(data_predictions[k]['score']))
        count += 1
        print(map_iou(np.array(data_labels[k]), np.array(data_predictions[k]['bbox']), np.array(data_predictions[k]['score'])))
print(sum/count)